In [ ]:
import glob
import pandas as pd
import joblib
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path_df = r"Table" + "\\"
path_model = r"Model" + "\\"
path_fig = r"Figure\Time Series" + "\\"

path_data = r"Data2" + "\\"
path_result0 = r"G:\2025-01-Arctic River Discharge Reconstruction\Product" + "\\"
path_result1 = path_result0 + r"01 River Discharge" + "\\"
path_result2 = path_result0 + r"02 Complete" + "\\"

col_list0 = ["River", "Station", "Temperature"]
col_list1 = [
    "River",
    "Station",
    "Temperature",
    "Temperature_Upper",
    "Temperature_Lower",
]
col_list2 = [
    "River",
    "Station",
    "Lon",
    "Lat",
    "Discharge_Station",
    "Discharge_Upper_Station",
    "Discharge_Lower_Station",
    "Source_Discharge",
    "Discharge_Estuary",
    "Discharge_Upper_Estuary",
    "Discharge_Lower_Estuary",
    "Temperature",
    "Temperature_Upper",
    "Temperature_Lower",
    # "Source_Temperature",
]

best_model = "XGBoost"

evade_list = [
    "Kem",
    "Khatanga",
    "Coppermine",
    "Thjorsa",
    "Lagarfljot",
    "Jokulsa A Fjollum",
    "Olfusa",
]

In [ ]:
sns.set_theme(style="white", font="Microsoft YaHei")

font1 = {"family": "Microsoft YaHei", "weight": "bold", "size": 16}

sources = ["Observed", "Predicted", "Interpolated", "Boundary matched"]
colors = ["blue", "green", "yellow", "red"]
color_dict = dict(zip(sources, colors))

# 模型训练与预测

In [ ]:
file_model = path_model + f"{best_model}_model.joblib"
model = joblib.load(file_model)

file_features = path_df + f"Selected_features_{best_model}.csv"
features = pd.read_csv(file_features).iloc[:, 0].tolist()

model

In [ ]:
X_train = pd.read_csv(path_df + "X_train.csv")[features]
y_train = pd.read_csv(path_df + "y_train.csv")

In [ ]:
params = model.get_params()

params["objective"] = "reg:quantileerror"
params["quantile_alpha"] = 0.975
model1 = XGBRegressor(**params)
model1.fit(X_train, y_train)

params["quantile_alpha"] = 0.025
model2 = XGBRegressor(**params)
model2.fit(X_train, y_train)

In [ ]:
file_list = glob.glob(path_data + "*.csv")
file_list = [file.split("\\")[-1] for file in file_list]
file_list.sort()
file_list = [file for file in file_list if file.split("__")[0] not in evade_list]
print(len(file_list))

for file in file_list[:]:
    print(file)
    df0 = pd.read_csv(path_result1 + file, parse_dates=["Date"], index_col="Date")

    df = pd.read_csv(path_data + file, parse_dates=["Date"], index_col="Date")
    river = df["River"].values[0]
    dates = df.index
    
    # Predict temperature
    X = df[features]
    df["Temperature"] = model.predict(X)
    df["Temperature"] = df["Temperature"].clip(lower=0)

    df["Temperature_Upper"] = model1.predict(X)
    df["Temperature_Upper"] = df["Temperature_Upper"].clip(lower=0)
    df["Temperature_Lower"] = model2.predict(X)
    df["Temperature_Lower"] = df["Temperature_Lower"].clip(lower=0)

    df = pd.merge(df, df0, on=["Date", "River", "Station"], how="left")
    df = df[col_list2]

    # Calculate heat flux
    df["Heat_Flux"] = (
        86400 * 1e6 * 4.184 * df["Discharge_Estuary"] * df["Temperature"] / 1e6
    )  # MJ/day
    df["Heat_Flux"] = df["Heat_Flux"].round(1)

    df["Heat_Flux_Upper"] = (
        86400
        * 1e6
        * 4.184
        * df["Discharge_Upper_Estuary"]
        * df["Temperature_Upper"]
        / 1e6
    )
    df["Heat_Flux_Upper"] = df["Heat_Flux_Upper"].round(1)

    df["Heat_Flux_Lower"] = (
        86400
        * 1e6
        * 4.184
        * df["Discharge_Lower_Estuary"]
        * df["Temperature_Lower"]
        / 1e6
    )
    df["Heat_Flux_Lower"] = df["Heat_Flux_Lower"].round(1)

    df.to_csv(path_result2 + file)